In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import pandas as pd
import torch
from PIL import Image
from transformers import ViltProcessor, ViltForQuestionAnswering

# === Normalization Utilities ===
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9", "ten": "10"
}

def normalize_answer(ans):
    ans = str(ans).strip().lower()
    if ans in number_map:
        return number_map[ans]
    if ans.isdigit():
        return ans
    if ans in ["yes", "no"]:
        return ans
    return ans

2025-05-14 01:48:42.210047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747187322.688182      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747187322.825857      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# === Model Setup ===
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)1qs`d

preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [8]:
# === Load Dataset ===
df = pd.read_csv("/kaggle/input/merged-vqa/merged_vqa_dataset_output.csv")
BASE_IMAGE_DIR = "/kaggle/input/abo-dataset/images/small"

all_results = []

In [9]:
# === Main Loop ===
for idx, row in df.iterrows():
    full_image_path = os.path.join(BASE_IMAGE_DIR, str(row['path']))
    try:
        image = Image.open(full_image_path).convert('RGB')
    except Exception as e:
        print(f"Failed to load image {full_image_path}: {e}")
        continue

    for q_col, a_col in zip(['q1', 'q2', 'q3'], ['a1', 'a2', 'a3']):
        question = row[q_col]
        gt_answer = row[a_col]

        try:
            inputs = processor(image, question, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                pred_idx = logits.argmax(-1).item()
                pred_answer = model.config.id2label[pred_idx]
        except Exception as e:
            print(f"Failed to predict for {full_image_path} / Q: {question} — {e}")
            pred_answer = "error"

        # Normalize for comparison
        gt_answer_norm = normalize_answer(gt_answer)
        pred_answer_norm = normalize_answer(pred_answer)
        is_correct = (pred_answer_norm == gt_answer_norm)

        all_results.append({
            "path": row['path'],
            "question": question,
            "gt_answer": gt_answer,
            "pred_answer": pred_answer,
            "is_correct": is_correct
        })

    if idx % 100 == 0:
        print(f"Processed {idx+1}/{len(df)} images")


Processed 1/15368 images
Processed 101/15368 images
Processed 201/15368 images
Processed 301/15368 images
Processed 401/15368 images
Processed 501/15368 images
Processed 601/15368 images
Processed 701/15368 images
Processed 801/15368 images
Processed 901/15368 images
Processed 1001/15368 images
Failed to predict for /kaggle/input/abo-dataset/images/small/5f/5f35a4d0.jpg / Q: What is in the image? — height and width must be > 0
Failed to predict for /kaggle/input/abo-dataset/images/small/5f/5f35a4d0.jpg / Q: What color is it? — height and width must be > 0
Failed to predict for /kaggle/input/abo-dataset/images/small/5f/5f35a4d0.jpg / Q: What is its shape? — height and width must be > 0
Processed 1101/15368 images
Processed 1201/15368 images
Processed 1301/15368 images
Processed 1401/15368 images
Processed 1501/15368 images
Processed 1601/15368 images
Processed 1701/15368 images
Processed 1801/15368 images
Processed 1901/15368 images
Processed 2001/15368 images
Processed 2101/15368 image

In [10]:
# === Final Evaluation ===
results_df = pd.DataFrame(all_results)
accuracy = results_df['is_correct'].mean()
print(f"\n✅ ViLBERT Baseline Accuracy: {accuracy:.3f}")

# === Save Output ===
results_df.to_csv("vilbert_vqa_baseline_results.csv", index=False)



✅ ViLBERT Baseline Accuracy: 0.258
